In [1]:
import os
import numpy as np
import ast
from order_fulfillment_environment import OrderFulfillment
from LP_fulfillment_item_facility_based import ItemFacilityLPSolver
from fulfillment_policy_Will_Ma import WillMaFulfillmentPolicy
from fulfillment_policy import FulfillmentPolicy

In [2]:
num_items = 20
n_max = 5
n_0 = 5
p_stock = 0.75
CSL=0.5
T = 10**2

instance = 1

facility_path = "fulfillment_centers_warmup.csv"
cities_path = "cities_warmup.csv"
home_path = "/Users/ayoubamil/Documents/Documents/GitHub/fulfillment-magician_AA_AM_YW/ORDER_FULFILLMENT/Data"
facilities_path = os.path.join(home_path, facility_path)
cities_path =  os.path.join(home_path, cities_path)

In [3]:
order_fulfillment = OrderFulfillment(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=CSL, 
                                    facilities_data=facilities_path, 
                                    cities_data=cities_path, 
                                    prob_seed_value=instance, 
                                    order_seed_value=instance, 
                                    inv_seed_value=instance)
fulfillment_policy = FulfillmentPolicy(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=CSL, 
                                    facilities_data=facilities_path, 
                                    cities_data=cities_path, 
                                    prob_seed_value=instance, 
                                    order_seed_value=instance, 
                                    inv_seed_value=instance)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-12


# CALCULATE DLP COSTS

In [4]:
def DLP_WillMa(order_fulfillment, print_optimal_values=False):
    
    solver = ItemFacilityLPSolver(order_fulfillment)

    results = solver.optimize()

    # Extract the results
    optimal_u = results["optimal_u"]
    optimal_y = results["optimal_y"]
    num_vars = results["num_vars"]
    num_constrs = results["num_constrs"]
    optimal_value = results["optimal_value"]

    # Step 4: Check if optimization was successful and print results
    if optimal_u and optimal_y:
        
        if print_optimal_values:
            print("Optimal 'u' values:")
            for var_name, value in optimal_u.items():
                print(f"{var_name} = {value}")

            print("\nOptimal 'y' values:")
            for var_name, value in optimal_y.items():
                print(f"{var_name} = {value}")

        return num_vars, num_constrs, optimal_value, optimal_u, optimal_y
    
    else:
        print("Optimization did not find an optimal solution for Will Ma's formulation.")

In [5]:
def DLP_ours(fulfillment_policy, print_optimal_values=False):
    # Call the method and get the results
    results = fulfillment_policy.get_optimization_results

    # Check if the optimization was successful and optionally print the results
    if results is not None:
        if print_optimal_values:
            print("Optimal 'x' values:")
            for q, x_values in results["x_values"].items():
                print(f"Order Type {q}:")
                for method_index, value in x_values.items():
                    print(f"  Method {method_index}: {value}")

            print(f"\nNumber of decision variables: {results['num_vars']}")
            print(f"Number of constraints: {results['num_constrs']}")
            print(f"Optimal objective value: {results['optimal_value']}")

        return results["num_vars"], results["num_constrs"], results["optimal_value"]
    
    else:
        print("Optimization did not find an optimal solution for our formulation.")
        return None

In [6]:
num_vars, num_constrs, optimal_value, optimal_u, optimal_y = DLP_WillMa(order_fulfillment)
num_vars, num_constrs, optimal_value

(32760, 5350, 1228.2835772124886)

In [7]:
DLP_ours(fulfillment_policy)

(164300, 360, 1257.7303335467377)

# CALCULATE ALGORITHM COSTS

In [22]:
conservative_prob = 0.25
seed_order_sequence = 2
inventory_consumption = fulfillment_policy.initialize_inventory_consumption()
magician_problems = fulfillment_policy.generate_magician_problems(conservative_prob=conservative_prob)

sampled_orders_index, sampled_orders, sampled_methods, accepts_decisions, fulfillment_costs = fulfillment_policy.new_fulfillment_policy(inventory_consumption, magician_problems, seed_value=seed_order_sequence)
fulfillment_policy.check_consistency(inventory_consumption)

our_cost = sum(fulfillment_costs)
our_cost

expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones


1599.099741198408

In [23]:
will_ma_policy = WillMaFulfillmentPolicy(order_fulfillment, optimal_u)
will_ma_cost = will_ma_policy.run(sampled_orders, seed_value=1)
will_ma_cost

1659.5079081659726